In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import datetime
import math
import re
from plot_utility import create_football_field, get_dx_dy, show_play
from string import punctuation
from sklearn.preprocessing import StandardScaler

In [3]:
def clean_wind_speed(windspeed):
    ws = str(windspeed)

    if 'mph' in ws.lower():
        return int(ws.lower().split('mph')[0])
    else :
        return ws

In [89]:
def preprocess(df):
    df=df.copy()
    
    #changing team abbr such that every team only has one unique abbr in the data
    df.loc[df.VisitorTeamAbbr == "ARI", 'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI", 'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL", 'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL", 'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE", 'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE", 'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU", 'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU", 'HomeTeamAbbr'] = "HST"
    
    #Stadium: mapping such that each arena only has one unique name
    stadium_map = {'Broncos Stadium at Mile High': 'Broncos Stadium At Mile High',
            'CenturyField': 'CenturyLink Field',
            'CenturyLink': 'CenturyLink Field',
            'Everbank Field': 'EverBank Field',
            'FirstEnergy': 'First Energy Stadium',
            'FirstEnergy Stadium': 'First Energy Stadium',
            'FirstEnergyStadium': 'First Energy Stadium',
            'Lambeau field': 'Lambeau Field',
            'Los Angeles Memorial Coliesum': 'Los Angeles Memorial Coliseum',
            'M & T Bank Stadium': 'M&T Bank Stadium',
            'M&T Stadium': 'M&T Bank Stadium',
            'Mercedes-Benz Dome': 'Mercedes-Benz Superdome',
            'MetLife': 'MetLife Stadium',
            'Metlife Stadium': 'MetLife Stadium',
            'NRG': 'NRG Stadium',
            'Oakland Alameda-County Coliseum': 'Oakland-Alameda County Coliseum',
            'Paul Brown Stdium': 'Paul Brown Stadium',
            'Twickenham': 'Twickenham Stadium'}
    
    df["Stadium"] = df['Stadium'].map(stadium_map).fillna(df["Stadium"])
    
    #Turf: map to Artificial/Natural
    turf_map = {'Field Turf':'Artificial',
        'A-Turf Titan':'Artificial',
        'Grass':'Natural',
        'UBU Sports Speed S5-M':'Artificial',
        'Artificial':'Artificial',
        'DD GrassMaster':'Artificial',
        'Natural Grass':'Natural',
        'UBU Speed Series-S5-M':'Artificial',
        'FieldTurf':'Artificial',
        'FieldTurf 360':'Artificial',
        'Natural grass':'Natural',
        'grass':'Natural',
        'Natural':'Natural',
        'Artifical':'Artificial',
        'FieldTurf360':'Artificial',
        'Naturall Grass':'Natural',
        'Field turf':'Artificial',
        'SISGrass':'Artificial',
        'Twenty-Four/Seven Turf':'Artificial',
        'natural grass':'Natural'}

    df['Turf'] = df['Turf'].map(turf_map).fillna(df["Turf"])
    
    #PlayerHeight: convert from feet+inches to inches
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))   
    
    #PlayerAge: compute player age as time between birthdate and timesnap
    df["PlayerAge"] = np.rint(((pd.to_datetime(df["TimeSnap"], format='%Y/%m/%d').dt.date -
                                pd.to_datetime(df["PlayerBirthDate"], format='%m/%d/%Y').dt.date)).dt.days / 365).astype("int")
    
    #GameClock: convert gameclock to time left of quarter
    df["GameClock"] = df["GameClock"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
    
    #StadiumType: Convert to Outdoor/Indoor
    stadiumType_map = {"Bowl": "Outdoor",
                   "Closed Dome": "Indoor",
                   "Cloudy": "Outdoor",
                   "Dome": "Outdoor",
                   "Dome, closed": "Indoor",
                   "Domed": "Indoor",
                   "Domed, open": "Outdoor",
                   "Domed, Open": "Outdoor",
                   "Domed, closed": "Indoor",
                   "Heinz Field": "Outdoor",
                   "Indoor, Open Roof": "Outdoor",
                   "Indoor, Roof Closed": "Indoor",
                   "Indoors": "Indoor",
                   "Open": "Outdoor",
                   "Oudoor": "Outdoor",
                   "Ourdoor": "Outdoor",
                   "Outdoor Retr Roof-Open": "Outdoor",
                   "Outdoors": "Outdoor",
                   "Outddors": "Outdoor",
                   "Outdor": "Outdoor",
                   "Outside": "Outdoor",
                   "Retr. Roof - Closed": "Indoor",
                   "Retr. Roof - Open": "Outdoor",
                   "Retr. Roof Closed": "Indoor",
                   "Retr. Roof-Closed": "Indoor",
                   "Retr. Roof-Open": "Outdoor",
                   "Retractable Roof": "Indoor"}
    
    df["StadiumType"] = df["StadiumType"].map(stadiumType_map).fillna(df["StadiumType"])
    #the three stadiums which contain NaN  are outdoor stadium
    df["StadiumType"].fillna("Outdoor", inplace = True)
    
    #GameWeather: map to rain/overcast/clear/snow/none
    rain = ['Rainy', 'Rain Chance 40%', 'Showers','Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
        'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain']

    overcast = ['Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
                'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
                'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
                'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
                'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
                'Partly Cloudy', 'Cloudy']

    clear = ['Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
            'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
            'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
            'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
            'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
            'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny', 'Sunny, Windy']

    snow  = ['Heavy lake effect snow', 'Snow']

    none  = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']
    
    df['GameWeather'] = df['GameWeather'].replace(rain,'rain')
    df['GameWeather'] = df['GameWeather'].replace(overcast,'overcast')
    df['GameWeather'] = df['GameWeather'].replace(clear,'clear')
    df['GameWeather'] = df['GameWeather'].replace(snow,'snow')
    df['GameWeather'] = df['GameWeather'].replace(none,'none')
    
    #WindSpeed: remove "mph" from strings and compute average if windspeed given in range
    df['WindSpeed'] = df['WindSpeed'].apply(clean_wind_speed)
    #impute 0 for WindSpeeds that does not make sense
    df['WindSpeed'] = df['WindSpeed'].replace(['nan','E','SE','Calm','SSW'], "0")
    
    #manually replace windspeeds given in ranges
    WindSpeed_map = {"11-17": "14",
                     "14-23": "18",
                     "12-22": "12",
                     "10-20": "15",
                     "15 gusts up to 25": "15"
                }
    df["WindSpeed"] = df["WindSpeed"].map(WindSpeed_map).fillna(df["WindSpeed"]).astype("int")
    
    #S: normalize S by season
    for season in df["Season"].unique():
        df.loc[train_df['Season'] == season, 'S'] = sc.fit_transform(df.loc[train_df['Season'] == season, 'S'].values.reshape(-1, 1))
    
    #Helper columns
    
    #TeamOnOffense: Home/away
    df["TeamOnOffense"] = np.where(df["PossessionTeam"] == df["HomeTeamAbbr"], "home", "away")
    
    #FieldPositionTeam: Side of the field, home/away
    df["FieldPositionTeam"] = np.where(df["FieldPosition"] == df["HomeTeamAbbr"], "home", "away")
    
    #IsOnOffense: True/False for attacking/defending
    df['IsOnOffense'] = df.Team == df.TeamOnOffense 
    
    #ScoreDiff
    df["ScoreDiff"] = np.where(df["TeamOnOffense"] == "home", df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"],
                               df["VisitorScoreBeforePlay"] - df["HomeScoreBeforePlay"])
    
    #Imputation missing values
    df["OffenseFormation"].fillna(df["OffenseFormation"].mode()[0], inplace = True)
    df["DefendersInTheBox"].fillna(df["DefendersInTheBox"].mode()[0], inplace = True)
    df["Temperature"].fillna(df["Temperature"].mode()[0], inplace = True)
    df["GameWeather"].fillna(df["GameWeather"].mode()[0], inplace = True)
    df["Humidity"].fillna(df["Humidity"].mode()[0], inplace = True)
    df["WindDirection"].fillna(df["WindDirection"].mode()[0], inplace = True)
    
    
    return df

In [90]:
def feature_engineering(df):
    
    #Create dataframe indexed on PlayId
    play_df = pd.DataFrame(df["PlayId"].unique(), columns = ["PlayId"])
    
    #Quarter
    feat_df = df.groupby("PlayId").agg({"Quarter": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #GameClock
    feat_df = df.groupby("PlayId").agg({"GameClock": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #Down
    feat_df = df.groupby("PlayId").agg({"Down": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #OffenseFormation
    feat_df = df.groupby("PlayId").agg({"OffenseFormation": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #DefendersInTheBox
    feat_df = df.groupby("PlayId").agg({"DefendersInTheBox": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #StadiumType
    feat_df = df.groupby("PlayId").agg({"StadiumType": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #Turf
    feat_df = df.groupby("PlayId").agg({"Turf": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #GameWeather
    feat_df = df.groupby("PlayId").agg({"GameWeather": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
 
    #Temperature
    feat_df = df.groupby("PlayId").agg({"Temperature": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #Humidity
    feat_df = df.groupby("PlayId").agg({"Humidity": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #ScoreDiff
    feat_df = df.groupby("PlayId").agg({"ScoreDiff": "first"}).reset_index()
    play_df = play_df.merge(feat_df, on = "PlayId", how = "left")
    
    #Create X
    play_df = play_df.merge(df[["PlayId", "Yards"]].drop_duplicates(), on = "PlayId", how = "left")
    play_df.set_index("PlayId", inplace = True)
    X = play_df.drop(columns = "Yards")
    
    #Create y
    yards = play_df["Yards"]
    y_matrix = np.zeros((yards.shape[0], 199))
    for idx, target in enumerate(list(yards)):
        y_matrix[idx][99 + target] = 1
    y = pd.DataFrame(y_matrix, columns = np.arange(-99, 100, 1))

    return X, y

In [81]:
def ohe(df):
    df = df.copy()
    for col in df.select_dtypes(include='object').columns:
        ohe_df = pd.get_dummies(df[col], prefix = col)
        df = df.merge(ohe_df, left_index = True, right_index = True, how = "left")
        df.drop(columns = col, inplace = True)
    return df

In [82]:
def scale_X(X):
    X = X.copy()
    scaler = StandardScaler()
    #only scale non binary columns
    scale_cols = X.apply(lambda x: x.nunique()).loc[lambda x : x != 2].index.tolist()
    non_scale_cols = [col for col in X.columns if col not in scale_cols]
    scaled_df  = pd.DataFrame(scaler.fit_transform(X[scale_cols]), columns = scale_cols, index = X.index)
    return pd.concat([X[non_scale_cols], scaled_df], axis = 1, sort = False)

In [83]:
%%time
train_df = pd.read_csv('train.csv', low_memory = False)
train_df.shape

CPU times: user 3.87 s, sys: 858 ms, total: 4.73 s
Wall time: 4.85 s


(509762, 49)

In [91]:
%%time
df = preprocess(train_df)

CPU times: user 5.19 s, sys: 294 ms, total: 5.48 s
Wall time: 5.5 s


In [92]:
%%time
X, y = feature_engineering(df)

CPU times: user 455 ms, sys: 139 ms, total: 594 ms
Wall time: 594 ms


In [93]:
%%time
X = ohe(X)

CPU times: user 30.5 ms, sys: 4.49 ms, total: 35 ms
Wall time: 33.8 ms


In [94]:
%%time
X = scale_X(X)

CPU times: user 18.2 ms, sys: 2.97 ms, total: 21.1 ms
Wall time: 20.2 ms


In [95]:
%%time
X.to_pickle("X")
y.to_pickle("y")

CPU times: user 25.2 ms, sys: 36.6 ms, total: 61.9 ms
Wall time: 62.8 ms
